In [58]:
from __future__ import absolute_import, division, print_function

from os import listdir
import xmltodict
import pandas
from StringIO import StringIO
%matplotlib inline
from tqdm import tqdm
import math
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from praelexis_file_system_provider.data_access_layer import AzureBlobStorageDataAccessLayer
import PyPDF2
import subprocess
import shutil
import os
from PIL import Image

### Connect to blob storage

In [59]:
blob_dal = AzureBlobStorageDataAccessLayer('devcapitecdata', 'Gt5z0EAAAm5ZETk//WBnRiQUZ9FDB/BmLmbx5ZPRtDwh6vxTDAB/XzLlM+m/epfnDYcZ8JDfMEGg1p9tBW0iFQ==', 'bank-statements', home_directory='drop/Documentum_line_files')

blob_dal.connect()
files = blob_dal.list(path='CI')

print ("Number of files: ", len(files))


Number of files:  8000


In [60]:
print(files[500].name)
print(files[501].name)
print(files[501].directory)

0f427a33-cbed-4673-a5d0-79dce1b8d45b.pdf
0f427a33-cbed-4673-a5d0-79dce1b8d45b_POCSTATEMENTSSTD_Capitec__19122017_022834_1154.xml
CI/


### Download and convert all pdfs to a text file

The software downloads each pdf and converts it to a tif file. The tif file is then sent through Tesseract. A text file is then generated for each pdf image that contains the estimated text found using Tesseract. The pdf and tif files are deleted immediately after they are not needed anymore, leaving only the text files.

In [ ]:
import commands
import time
#import glob

folderDir = "./test_files/"
%env TESSDATA_PREFIX=/tesseract/tessdata
count = 0
for fileNum in tqdm(range(len(files))):
    if files[fileNum].name[-3:]=="pdf":
        count += 1
        pdfName = files[fileNum].directory + files[fileNum].name
        if not os.path.isfile(folderDir+"Example"+str(count)+".box"):
            print("Working on file number: "+str(count))
            # Download the pdf file
            start = time.time()
            matchString = files[fileNum].name[:-4]
            stream = StringIO()
            blob_dal.get(pdfName, stream)
            pdfDir = folderDir+"temp.pdf"
            with open (pdfDir, 'w') as tmp:
              stream.seek (0)
              shutil.copyfileobj (stream, tmp)
            print("It took ",round(time.time()-start,2)," seconds download the pdf.")
            
            # Convert the pdf to tif and then to text
            
            start = time.time()
            tifDir = folderDir+"temp.tif"
            s=commands.getstatusoutput("convert -density 300 -units pixelsperinch "+pdfDir+" -quality 100 "+tifDir)
            print(s)
            print("It took ",round(time.time()-start,2)," seconds to convert the pdf to tif.")
            
            print("Deleting the pdf file.")
            s=commands.getstatusoutput("rm " + pdfDir)
            print("Pdf deleted.")
            
            start = time.time()
            command = "tesseract " + tifDir + " "+folderDir+"Example"+str(count)+" -l eng batch.nochop makebox"
            print("Command: ", command)
            s=commands.getstatusoutput(command)
            print(s)
            print("It took ",round(time.time()-start), " to run tesseract on the tif file.")

            print("Deleting the tif file.")
            s=commands.getstatusoutput("rm " + tifDir)
            print("Tif deleted.")

  0%|          | 0/8000 [00:00<?, ?it/s]

env: TESSDATA_PREFIX=/tesseract/tessdata
Working on file number: 1
It took  0.16  seconds download the pdf.
(0, '')
It took  9.34  seconds to convert the pdf to tif.
Deleting the pdf file.
Pdf deleted.
Command:  tesseract ./test_files/temp.tif ./test_files/Example1 -l eng batch.nochop makebox


  0%|          | 1/8000 [00:53<117:51:13, 53.04s/it]

(0, 'Tesseract Open Source OCR Engine v4.00.00alpha with Leptonica\nPage 1\nPage 2\nPage 3\nPage 4\nPage 5\nPage 6')
It took  43.0  to run tesseract on the tif file.
Deleting the tif file.
Tif deleted.
Working on file number: 2
It took  0.07  seconds download the pdf.
(0, '')
It took  3.61  seconds to convert the pdf to tif.
Deleting the pdf file.
Pdf deleted.
Command:  tesseract ./test_files/temp.tif ./test_files/Example2 -l eng batch.nochop makebox


  0%|          | 3/8000 [01:09<88:01:36, 39.63s/it] 

...

(0, 'Tesseract Open Source OCR Engine v4.00.00alpha with Leptonica\nPage 1\nPage 2\nPage 3\nPage 4\nPage 5\nPage 6\nPage 7\nPage 8')
It took  60.0  to run tesseract on the tif file.
Deleting the tif file.
Tif deleted.
Working on file number: 4
It took  0.1  seconds download the pdf.
(0, '')
It took  19.16  seconds to convert the pdf to tif.
Deleting the pdf file.
Pdf deleted.
Command:  tesseract ./test_files/temp.tif ./test_files/Example4 -l eng batch.nochop makebox


  0%|          | 7/8000 [04:17<98:17:10, 44.27s/it]

(0, 'Tesseract Open Source OCR Engine v4.00.00alpha with Leptonica\nPage 1\nPage 2\nPage 3\nPage 4\nPage 5\nPage 6\nPage 7\nPage 8\nPage 9\nPage 10\nPage 11\nPage 12')
It took  95.0  to run tesseract on the tif file.
Deleting the tif file.
Tif deleted.
Working on file number: 5
It took  0.12  seconds download the pdf.
(0, '')
It took  11.76  seconds to convert the pdf to tif.
Deleting the pdf file.
Pdf deleted.
Command:  tesseract ./test_files/temp.tif ./test_files/Example5 -l eng batch.nochop makebox


  0%|          | 9/8000 [05:19<89:34:37, 40.36s/it]

(0, 'Tesseract Open Source OCR Engine v4.00.00alpha with Leptonica\nPage 1\nPage 2\nPage 3\nPage 4\nPage 5\nPage 6\nPage 7')
It took  50.0  to run tesseract on the tif file.
Deleting the tif file.
Tif deleted.
Working on file number: 6
It took  0.18  seconds download the pdf.
